# Genex Core Agents — Profile Memory + Research + Therapy Planning (ADK Notebook)

## What this notebook is
This notebook is the **core Genex agent prototype** built with **Google ADK (Agent Development Kit)**. It demonstrates how **persistent child context** (profile memory) can be shared across **multiple agent types**—research, planning, and summarization—within a single conversational session.

Think of this file as the **foundation layer** from which the more specialized Genex notebooks (therapy planners, Q&A pipelines, research-only agents) are derived.

> This is an educational prototype and **not a medical system**.

---

## Inputs (what you must provide)
### 1) API keys
Depending on which agents are enabled, this notebook may initialize:
- `OPENAI_API_KEY` (GPT)
- `GOOGLE_API_KEY` (Gemini)
- `ANTHROPIC_API_KEY` (Claude)

Keys are loaded via `python-dotenv`.

### 2) Parent prompts (runtime)
You interact with the system through:
- an initial message containing **child name, age, diagnosis**
- follow-up messages requesting research, activities, or plans

All interactions must use the same `user_id` and `session_id` to preserve context.

---

## What it does (process overview)
### A) Child profile memory (shared state)
- A **Profile Agent** extracts and stores child metadata in ADK session state.
- A retrieval tool ensures downstream agents can safely access the profile before responding.

This enables:
- “Tell me more activities” without repeating the child’s info
- Cross-agent continuity in the same session

### B) Modular agents (composable)
This notebook defines and wires together multiple agent roles, typically including:
- **Research Agent** – therapy or intervention research based on diagnosis and age
- **Planning Agent** – structured suggestions (e.g., daily activities or routines)
- **Summary / Response Agent** – parent-friendly explanations and recap

Agents may run sequentially or conditionally depending on the prompt.

### C) Single conversational runner
A single ADK `Runner` orchestrates:
- profile storage (if missing)
- agent invocation
- final response emission

---

## Outputs (what you get)
### Console output
- Each call prints the final agent response (research, plan, or summary).
- Intermediate tool calls can be viewed when using `run_debug()`.

### Session state (in-memory)
Using `InMemorySessionService`, the notebook stores:
- `child_profile`
- optional intermediate artifacts (research notes, plan drafts, etc.)

State persists only for the life of the kernel and the chosen session id.

---

## How to run
1) Run all cells top-to-bottom to register tools, agents, and the runner.
2) Start with a prompt that includes the child profile.
3) Continue the conversation with follow-up requests (research, activities, refinements).

---

## When to use this notebook
- As a **reference architecture** for Genex agents
- For experimentation with shared memory across agents
- As a starting point for building new pipelines (therapy, education, care coordination)

## Limitations / known constraints
- In-memory state only (no database persistence).
- Behavior depends on enabled models and API access.
- Not intended for diagnosis or clinical decision-making.


In [ ]:
# Authentication
import os
from dotenv import load_dotenv

load_dotenv()  # loads .env from project root

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

from openai import OpenAI
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

from anthropic import Anthropic
anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

print("All API clients initialized successfully.")
print("Google:", os.getenv("GOOGLE_API_KEY")[:5])
print("OpenAI:", os.getenv("OPENAI_API_KEY")[:5])
print("Anthropic:", os.getenv("ANTHROPIC_API_KEY")[:5])

In [2]:
# Imports
from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner, Runner
from google.adk.tools import FunctionTool, ToolContext
from google.adk.sessions import DatabaseSessionService
from google.adk.apps.app import App, EventsCompactionConfig
from google.genai import types
from openai import OpenAI
from anthropic import Anthropic
import uuid

from typing import Any, Dict


In [3]:
# Gemini test
test_agent = LlmAgent(
    name="test",
    model=Gemini(model="gemini-2.5-flash"),
    instruction="Say hi."
)
runner = InMemoryRunner(agent=test_agent)
await runner.run_debug("Hello")


 ### Created new session: debug_session_id

User > Hello
test > Hi! How can I help you today?


[Event(model_version='gemini-2.5-flash', content=Content(
   parts=[
     Part(
       text='Hi! How can I help you today?'
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=9,
   prompt_token_count=19,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=19
     ),
   ],
   thoughts_token_count=16,
   total_token_count=44
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-965de6a4-a4c0-4917-b42d-271a082df5cc', author='test', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None

In [4]:
# OpenAI test
openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role":"user","content":"hi"}]
)

ChatCompletion(id='chatcmpl-Cgy97neHJwn66g9rA9WfhfO9d2Ruw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1764357013, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [5]:
# Anthropic test
anthropic_client.messages.create(
    model="claude-3-5-haiku-20241022",
    max_tokens=10,
    messages=[{"role":"user","content":"hi"}]
)

Message(id='msg_017jLf6cb3S5DTAVQM9dYLGB', content=[TextBlock(citations=None, text='Hello! How are you doing today? Is there', type='text')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='max_tokens', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=8, output_tokens=10, server_tool_use=None, service_tier='standard'))

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

### Helper Function Save and Retrieve Child's Profile

In [8]:
def save_child_profile(
    tool_context: ToolContext,
    name: str,
    age_years: int,
    diagnosis: str,
) -> Dict[str, Any]:
    """
    Store child profile (e.g., Emma, 3, Down syndrome) in session state.
    """
    tool_context.state["child:name"] = name
    tool_context.state["child:age_years"] = age_years
    tool_context.state["child:diagnosis"] = diagnosis

    return {"status": "success"}


def retrieve_child_profile(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Retrieve child profile from session state.
    """
    name = tool_context.state.get("child:name")
    age_years = tool_context.state.get("child:age_years")
    diagnosis = tool_context.state.get("child:diagnosis")

    if name is None or age_years is None or diagnosis is None:
        return {
            "status": "error",
            "message": "No child profile found in this session.",
        }

    return {
        "status": "success",
        "name": name,
        "age_years": age_years,
        "diagnosis": diagnosis,
    }

### GPT Research Tool 

In [9]:
def gpt_child_research_tool(tool_context: ToolContext) -> dict:
    """
    Uses Emma's profile from session state to call GPT-4o-mini
    and get age- & diagnosis-specific therapy recommendations.
    """
    profile = retrieve_child_profile(tool_context)
    if profile["status"] != "success":
        return {
            "status": "error",
            "error_message": "No child profile found. Please provide name, age, and diagnosis first."
        }

    name = profile["name"]
    age = profile["age_years"]
    diagnosis = profile["diagnosis"]

    prompt = f"""
    You are a pediatric developmental specialist.

    Child profile:
    - Name: {name}
    - Age: {age} years
    - Diagnosis: {diagnosis}

    Task:
    1. Give a one-sentence summary of {diagnosis} in the context of a {age}-year-old child.
    2. Provide recommended therapies tailored SPECIFICALLY for this child's age and condition, organized under:
       - language/communication
       - physical/movement
       - occupational/fine motor
       - social/emotional
       - cognitive/learning
    3. Phrase everything as advice to the parents of {name}.
    """

    result = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
    )

    return {
        "status": "success",
        "gpt_research": result.choices[0].message.content,
    }


### Claude Research Tool

In [10]:
def claude_child_research_tool(tool_context: ToolContext) -> dict:
    """
    Uses Emma's profile from session state to call Claude 3.5 Haiku.
    """
    profile = retrieve_child_profile(tool_context)
    if profile["status"] != "success":
        return {
            "status": "error",
            "error_message": "No child profile found. Please provide name, age, and diagnosis first."
        }

    name = profile["name"]
    age = profile["age_years"]
    diagnosis = profile["diagnosis"]

    prompt = f"""
    You are a pediatric developmental specialist.

    Child profile:
    - Name: {name}
    - Age: {age} years
    - Diagnosis: {diagnosis}

    Task:
    1. Give a one-sentence summary of {diagnosis} in the context of a {age}-year-old child.
    2. Provide recommended therapies tailored SPECIFICALLY for this child's age and condition, organized under:
       - language/communication
       - physical/movement
       - occupational/fine motor
       - social/emotional
       - cognitive/learning
    3. Phrase everything as advice to the parents of {name}.
    """

    response = anthropic_client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=700,
        messages=[{"role": "user", "content": prompt}],
    )

    claude_text = "".join(block.text for block in response.content)

    return {
        "status": "success",
        "claude_research": claude_text,
    }

### Wrappers for GPT and Claude 

In [11]:
gpt_tool = FunctionTool(func=gpt_child_research_tool)
claude_tool = FunctionTool(func=claude_child_research_tool)

### GPT Research Agent

In [12]:
gpt_researcher = LlmAgent(
    name="gpt_researcher",
    model=Gemini(model="gemini-2.5-flash"),
    instruction="Always call gpt_child_research_tool; do not answer directly.",
    tools=[gpt_tool],
    output_key="gpt_research",
)

### Claude Research Agent 

In [13]:
claude_researcher = LlmAgent(
    name="claude_researcher",
    model=Gemini(model="gemini-2.5-flash"),
    instruction="Always call claude_child_research_tool; do not answer directly.",
    tools=[claude_tool],
    output_key="claude_research",
)

### Gemini Research Agent 

In [14]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import FunctionTool

# If you haven't wrapped retrieve_child_profile yet:
retrieve_child_profile_tool = FunctionTool(func=retrieve_child_profile)

gemini_researcher = LlmAgent(
    name="gemini_researcher",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    instruction="""
You are a pediatric developmental specialist and part of a multi-model research team.

Your behavior:

1. First, call the `retrieve_child_profile` tool to get:
   - child's name
   - age in years
   - diagnosis / condition

2. If the tool returns an error (no profile found), respond with a short,
   clear message explaining that you cannot proceed without:
   - name
   - age
   - diagnosis.

3. If the profile is found, write a tailored answer for the parents, using this structure:

   - Start with: "For [NAME], [AGE] years old with [DIAGNOSIS], ..."
   - Then provide recommendations organized under:
       * language & communication
       * physical & movement
       * occupational & fine-motor
       * social & emotional
       * cognitive & learning

   - Make sure the recommendations are appropriate for the child's age
     (e.g., play-based, early intervention focus for a 3-year-old).

4. Be concise but specific, and write in a warm, supportive tone for the parents.

Your final answer will be saved as `gemini_research` and consumed by a downstream aggregator.
""",
    tools=[retrieve_child_profile_tool],
    output_key="gemini_research",
)

### Profile agent

Ensures Emma is in state at the start of the pipeline. We keep a profile agent that stores Emma:

In [15]:
profile_agent = LlmAgent(
    name="profile_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Collects and stores child profile info in session state.",
    instruction="""
When the user describes their child (name, age, condition/diagnosis),
you MUST call `save_child_profile` to store those details in session state.

If the child profile is already stored and the user updates info
(e.g., a new age or additional diagnoses), call `save_child_profile` again.

Do NOT give therapy advice yourself.
""",
    tools=[save_child_profile, retrieve_child_profile],
)

### Root pipeline
#### profile → parallel research → aggregator

In [16]:
parallel_research_team = ParallelAgent(
    name="parallel_research_team",
    sub_agents=[gemini_researcher, gpt_researcher, claude_researcher],
)

aggregator_agent = LlmAgent(
    name="aggregator_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    tools=[retrieve_child_profile_tool],
    instruction="""
First, call `retrieve_child_profile` to get:
- child name
- age in years
- diagnosis

You are the final summarizer. You MUST produce exactly this structure in your answer:

1) First line:
   "Emma is [AGE] years old and has [DIAGNOSIS]."
   Use the actual name, age, and diagnosis from the profile.

2) Second line:
   A single, clear sentence:
   "Down syndrome is ..."

3) Then, a section titled:
   "Therapy Recommendations by Category"
   Under this, create subsections:

   - **Language & Communication**
     One concise paragraph summarizing what all three researchers agree on.
   - **Physical & Movement**
     One concise paragraph summarizing agreements.
   - **Occupational & Fine Motor**
     One concise paragraph summarizing agreements.
   - **Social & Emotional**
     One concise paragraph summarizing agreements.
   - **Cognitive & Learning**
     One concise paragraph summarizing agreements.

   These summaries MUST be based on:

   Gemini research:
   {gemini_research}

   GPT research:
   {gpt_research}

   Claude research:
   {claude_research}

4) Finally, add a section:
   "Differences Between Researchers"
   - If they largely agree, say that in one short sentence.
   - If there are differences (e.g., one suggests more professional therapy, another emphasizes home-based play), list 2–4 bullet points describing the key differences.

Tone: warm, clear, and parent-friendly. Do NOT repeat full long lists; only summarize.
""",
    output_key="final_summary",
)

root_agent = SequentialAgent(
    name="emma_research_system",
    sub_agents=[profile_agent, parallel_research_team, aggregator_agent],
)

### Persistent sessions per child / family

In [17]:
APP_NAME = "genex_research_app"
USER_ID = "emma_parents"
DB_URL = "sqlite:///genex_sessions.db"

session_service = DatabaseSessionService(db_url=DB_URL)

research_app = App(
    name=APP_NAME,
    root_agent=root_agent,
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=5,
        overlap_size=1,
    ),
)

runner = Runner(app=research_app, session_service=session_service)

C:\Users\T490\AppData\Local\Temp\ipykernel_19468\1568630742.py:10: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


#### First time

In [18]:
await runner.run_debug(
    "This is about my daughter Emma. She is 3 years old and has Down syndrome.",
    user_id="emma_parent",
    session_id="emma-session-001",
    quiet=True,
    verbose=False
)

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'age_years': 3,
           'diagnosis': 'Down syndrome',
           'name': 'Emma'
         },
         id='adk-ee763f81-117a-47e6-9cc5-1afd56130504',
         name='save_child_profile'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=30,
   prompt_token_count=249,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=249
     ),
   ],
   total_token_count=279
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=N

In [19]:
events = await runner.run_debug(
    "What therapies do you recommend for her overall development?",
    user_id="emma_parent",
    session_id="emma-session-001",
    quiet=True,
    verbose=False
)

In [20]:
final_event = events[-1]  # last event comes from aggregator agent
final_summary = final_event.actions.state_delta["final_summary"]

print(final_summary)

Emma is 3 years old and has Down syndrome.
Down syndrome is a genetic condition caused by an extra copy of chromosome 21 that affects physical development, cognitive abilities, and learning.
Therapy Recommendations by Category
**Language & Communication**
All three researchers emphasize using simple language, short sentences, and visual aids like picture cards to support understanding and expression. They all recommend interactive reading and agree on the benefits of encouraging both verbal and non-verbal communication. Gemini and Claude specifically suggest sign language as a helpful tool.

**Physical & Movement**
There is a strong consensus among the researchers on the importance of active play for developing gross motor skills, coordination, and balance. Recommended activities include climbing, dancing, and playing with balls. Gemini also suggests crawling through tunnels, while Claude mentions swimming and walking on various surfaces. Claude specifically recommends consulting with 

#### Later

In [21]:
await runner.run_debug(
    "Can you suggest more fine-motor activities for Emma?",
    user_id=USER_ID,
    session_id="emma-session-001",
)


 ### Continue session: emma-session-001

User > Can you suggest more fine-motor activities for Emma?
profile_agent > Here are some additional fine-motor activities for Emma:

*   **Sensory Bins:** Fill a bin with rice, beans, sand, or water beads and provide scoops, cups, and small toys for her to explore. This helps with grasping, pouring, and scooping.
*   **Play with Water:** Activities like transferring water between containers with sponges or droppers, or playing with bath toys that squirt water, can be very engaging and beneficial for fine motor control.
*   **Stickers:** Peeling and sticking stickers onto paper or designated spots can help develop pincer grasp and hand-eye coordination. Start with larger stickers and move to smaller ones as she progresses.
*   **Crumpling Paper:** Give Emma newspaper or tissue paper to crumple into balls. This is a great way to build hand strength.
*   **Using Tongs:** Provide child-safe tongs or tweezers for her to pick up small objects like p

gemini_researcher > For Emma, 3 years old with Down syndrome, here are some more fine-motor activities to support her development:

*   **Manipulating Fasteners:** Introduce activity boards or old clothes with large buttons, zippers, and velcro closures for her to practice opening and closing. This helps with dexterity and preparing for self-dressing.
*   **Simple Snipping with Child-Safe Scissors:** With supervision, provide blunt-nosed child scissors and strips of paper for her to practice making snips. This builds hand strength and coordination.
*   **Inserting Objects into Slots:** Offer a piggy bank with coins (large, safe ones) or a container with a slit for her to push craft sticks or cards through. This refines precision and hand-eye coordination.
*   **Peeling and Separating:** Encourage her to peel easy-to-manage items like a banana (after you start it for her) or pull apart orange segments. This uses both hands and develops finger strength.
*   **Small Construction Toys:** B

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       text="""Here are some additional fine-motor activities for Emma:
 
 *   **Sensory Bins:** Fill a bin with rice, beans, sand, or water beads and provide scoops, cups, and small toys for her to explore. This helps with grasping, pouring, and scooping.
 *   **Play with Water:** Activities like transferring water between containers with sponges or droppers, or playing with bath toys that squirt water, can be very engaging and beneficial for fine motor control.
 *   **Stickers:** Peeling and sticking stickers onto paper or designated spots can help develop pincer grasp and hand-eye coordination. Start with larger stickers and move to smaller ones as she progresses.
 *   **Crumpling Paper:** Give Emma newspaper or tissue paper to crumple into balls. This is a great way to build hand strength.
 *   **Using Tongs:** Provide child-safe tongs or tweezers for her to pick up small objects like pom-poms or c

[Profile Agent] → stores child:name, age, diagnosis

[Parallel Research Team]
   - ├── Gemini researcher → Gemini_child_tool → personalized research
   - ├── GPT researcher    → GPT_child_tool    → personalized research
   - └── Claude researcher → Claude_child_tool → personalized research

[Aggregator Agent] → merges 3 sources using child profile


### DB Inspection

In [27]:
import sqlite3

def print_events_schema(db_path="genex_sessions.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    print("🔍 EVENTS TABLE SCHEMA:")
    rows = cursor.execute("PRAGMA table_info(events);")
    for r in rows.fetchall():
        print(r)

    conn.close()

print_events_schema()


🔍 EVENTS TABLE SCHEMA:
(0, 'id', 'VARCHAR(128)', 1, None, 1)
(1, 'app_name', 'VARCHAR(128)', 1, None, 2)
(2, 'user_id', 'VARCHAR(128)', 1, None, 3)
(3, 'session_id', 'VARCHAR(128)', 1, None, 4)
(4, 'invocation_id', 'VARCHAR(256)', 1, None, 0)
(5, 'author', 'VARCHAR(256)', 1, None, 0)
(6, 'actions', 'BLOB', 1, None, 0)
(7, 'long_running_tool_ids_json', 'TEXT', 0, None, 0)
(8, 'branch', 'VARCHAR(256)', 0, None, 0)
(9, 'timestamp', 'DATETIME', 1, None, 0)
(10, 'content', 'TEXT', 0, None, 0)
(11, 'grounding_metadata', 'TEXT', 0, None, 0)
(12, 'custom_metadata', 'TEXT', 0, None, 0)
(13, 'usage_metadata', 'TEXT', 0, None, 0)
(14, 'citation_metadata', 'TEXT', 0, None, 0)
(15, 'partial', 'BOOLEAN', 0, None, 0)
(16, 'turn_complete', 'BOOLEAN', 0, None, 0)
(17, 'error_code', 'VARCHAR(256)', 0, None, 0)
(18, 'error_message', 'VARCHAR(1024)', 0, None, 0)
(19, 'interrupted', 'BOOLEAN', 0, None, 0)
